In [1]:
import functions as func
import operator
import pandas as pd
import numpy as np
import re

In [2]:
term2POS = func.readDictBig('external/dict.txt.big')

In [3]:
polarity_contents = func.parsePolarityOut('data/polarity_simpl.out', term2POS)

aspect_contents = func.parseAspectOut('data/aspect_simpl.out', term2POS)

testReview_contents = func.parseTestReviewOut('data/test_simpl.out', term2POS)

In [4]:
contents = polarity_contents+aspect_contents+testReview_contents
model = func.train_word2vec_model(contents)

In [5]:
serviceTerms = ['服务','态度','人员']
envTerms = ['环境','客房','设备','空调']
priceTerms = ['价格','房价']
trafficTerms = ['交通','地理']
restaurantTerms = ['餐厅','早餐']

In [6]:
thre = 0.4
serviceNewTerms = func.getSimTerms(model, serviceTerms, [0.4, 0.4, 0.4])
envNewTerms = func.getSimTerms(model, envTerms, [0.4, 0.4, 0.4, 0.4])
priceNewTerms = func.getSimTerms(model, priceTerms, [0.4, 0.4])
trafficNewTerms = func.getSimTerms(model, trafficTerms, [0.4, 0.4])
restaurantNewTerms = func.getSimTerms(model, restaurantTerms, [0.4, 0.4])

In [7]:
print(len(serviceNewTerms))
print(len(envNewTerms))
print(len(priceNewTerms))
print(len(trafficNewTerms))
print(len(restaurantNewTerms))

#print(serviceNewTerms)
#print(envNewTerms)
#print(priceNewTerms)
#print(trafficNewTerms)
#print(restaurantNewTerms)

409
697
248
239
312


In [8]:
testReviewDict = func.readTestReview('data/test_simpl.out')
testDF = pd.read_csv('data/test.csv')

In [ ]:
f = open('Aspect_per_sentence.csv', 'w')
for Id, comment in testReviewDict.items():
    sentences = re.split('[,.～！，。；]', comment)
    for sentence in sentences:
        f.write(str(Id)+","+str(func.decideAspect(sentence, serviceNewTerms, envNewTerms, priceNewTerms, trafficNewTerms, restaurantNewTerms))+'\n')
f.close()

In [14]:
f = open('result.csv', 'w')
f.write('Id,Label\n')
for i in range(testDF.shape[0]):
    Id, Review_id, Aspect = testDF.iloc[i]
    f.write(str(Id)+","+str(func.countLabel(testReviewDict[int(Review_id)], str(Aspect), serviceNewTerms, envNewTerms, priceNewTerms, trafficNewTerms, restaurantNewTerms))+'\n')
f.close()